In [1]:
import pandas as pd

Cоздайте представления dataframe с двумя столбцами: datetime и user, прочитав feed-views.log
- преобразовать дату и время в тип datetime64[ns]
- извлечь год, месяц, день, час, минуту и ​​секунду из значений этого столбца в новые столбцы

In [2]:
df = pd.read_csv(
    'data/feed-views.log',
    sep='\t',
    skiprows=[2,3],
    skipfooter=2,
    names=['datetime','user'],
    engine='python',
)
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second
df.set_index('datetime', inplace=True)
print(df.head())
print('')
print(df.tail())

                              user  year  month  day  hour  minute  second
datetime                                                                  
2020-04-17 12:01:08.463179   artem  2020      4   17    12       1       8
2020-04-17 12:01:23.743946   artem  2020      4   17    12       1      23
2020-04-17 12:35:52.735016   artem  2020      4   17    12      35      52
2020-04-17 12:36:21.401412  oksana  2020      4   17    12      36      21
2020-04-17 12:36:22.023355  oksana  2020      4   17    12      36      22

                                 user  year  month  day  hour  minute  second
datetime                                                                     
2020-05-21 16:36:40.915488  ekaterina  2020      5   21    16      36      40
2020-05-21 17:49:36.429237      maxim  2020      5   21    17      49      36
2020-05-21 18:45:20.441142  valentina  2020      5   21    18      45      20
2020-05-21 23:03:06.457819      maxim  2020      5   21    23       3       6
2020-0

Cоздать новую колонку дневное время
- присвоить конкретное значение времени суток, если час находится в определенном интервале, например, полдень, если час больше 11 и меньше или равен 17
- 0 – 3.59 ночь, 4 – 6.59 раннее утро, 7 – 10.59 утро, 11 – 16.59 день, 17 – 19.59 ранний вечер, 20 – 23.59 вечер
- используйте метод cut для решения этой подзадачи
- назначить столбец user в качестве индекса

In [3]:
times = [-1, 3.99, 6.99, 10.99, 16.99, 19.99, 23.99]
times_str = ['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening']
df['daytime'] = pd.cut(df['hour'], bins=times, labels=times_str, right=True)
df.set_index('user', inplace=True)
print(df.head())
print('')
print(df.tail())

        year  month  day  hour  minute  second    daytime
user                                                     
artem   2020      4   17    12       1       8  afternoon
artem   2020      4   17    12       1      23  afternoon
artem   2020      4   17    12      35      52  afternoon
oksana  2020      4   17    12      36      21  afternoon
oksana  2020      4   17    12      36      22  afternoon

           year  month  day  hour  minute  second        daytime
user                                                            
ekaterina  2020      5   21    16      36      40      afternoon
maxim      2020      5   21    17      49      36  early evening
valentina  2020      5   21    18      45      20  early evening
maxim      2020      5   21    23       3       6        evening
pavel      2020      5   21    23      23      49        evening


Подсчитайте количество элементов в вашем фрейме данных
- используйте метод count()
- подсчитать количество элементов в каждой категории времени суток, используя метод value_counts()

In [4]:
daytime_count = df.count()
daytime_counts = df['daytime'].value_counts()
print(daytime_count)
print(' ')
print(daytime_counts)

year       1072
month      1072
day        1072
hour       1072
minute     1072
second     1072
daytime    1072
dtype: int64
 
daytime
evening          508
afternoon        250
early evening    145
night            129
morning           35
early morning      5
Name: count, dtype: int64


Сортировать значения в вашем фрейме данных по часам, минутам и секундам в порядке возрастания

In [5]:
df_sorted = df.sort_values(by=['hour', 'minute', 'second'], ascending=True)
df_sorted

,year,month,day,hour,minute,second,daytime
user,,,,,,,
valentina,2020,5,15,0,0,13,night
valentina,2020,5,15,0,1,5,night
pavel,2020,5,12,0,1,27,night
pavel,2020,5,12,0,1,38,night
pavel,2020,5,12,0,1,55,night
...,...,...,...,...,...,...,...
artem,2020,4,29,23,48,14,evening
anatoliy,2020,5,9,23,53,55,evening
pavel,2020,5,9,23,54,54,evening


Рассчитать минимум и максимум для часов и режим для дневных категорий
- вычислить максимальный час для строк, где время суток — ночь
- вычислить минимальный час для строк, где время суток — утро
- В дополнение к этому, выясните, кто посещал страницу в эти часы (приведите один пример из этого)
- рассчитать моду на час и день

In [6]:
max_hour_night = df[df['daytime'] == 'night']['hour'].max()
print(f'Максимальный час ночью: {max_hour_night}')

Максимальный час ночью: 3


In [7]:
min_hour_morning = df[df['daytime'] == 'morning']['hour'].min()
print(f'Минимальный час утром: {min_hour_morning}')

Минимальный час утром: 8


In [8]:
visitors_night = df[(df['daytime'] == 'night') & (df['hour'] == max_hour_night)].index.unique()
visitors_morning = df[(df['daytime'] == 'morning') & (df['hour'] == min_hour_morning)].index.unique()
print(f'Посещал ночью: {visitors_night[0]}')
print(f'Посещал утром: {visitors_morning[0]}')

Посещал ночью: konstantin
Посещал утром: alexander


In [9]:
mode_hour = df['hour'].mode()[0]
mode_daytime = df['daytime'].mode()[0]
print(f'Наиболее часто встречающееся время в часах: {mode_hour}')
print(f'Наиболее часто встречающееся время суток: {mode_daytime}')

Наиболее часто встречающееся время в часах: 22
Наиболее часто встречающееся время суток: evening


Показать 3 самых ранних часа утра и соответствующие им имена пользователей, а также 3 самых поздних часа и имена пользователей с помощью nsmallest() и nlargest()

In [10]:
earliest_morning = df[df['daytime'] == 'morning'].nsmallest(3, 'hour')
latest_hours = df.nlargest(3, 'hour')
earliest_hours_and_users = [(user, hour) for user, hour in zip(earliest_morning.index, earliest_morning['hour'])]
latest_hours_and_users = [(user, hour) for user, hour in zip(latest_hours.index, latest_hours['hour'])]

print('Самые ранние утренние часы и пользователи:')
for user, hour in earliest_hours_and_users:
    print(f'{hour} - {user}')

print('\nСамые поздние часы и пользователи:')
for user, hour in latest_hours_and_users:
    print(f'{hour} - {user}')

Самые ранние утренние часы и пользователи:
8 - alexander
8 - alexander
9 - artem

Самые поздние часы и пользователи:
23 - konstantin
23 - artem
23 - artem


Используйте метод describe() для получения базовой статистики по столбцам
- чтобы узнать, какой интервал посещения страницы является наиболее популярным, рассчитайте межквартильный размах для часа, извлекая значения из результата метода describe() и сохраняя его в переменной iqr

In [11]:
hour_stats = df['hour'].describe()
q3 = hour_stats['75%']
q1 = hour_stats['25%']
iqr = q3 - q1
print(f'Мeжквартильный размах (IQR): {iqr}')

Мeжквартильный размах (IQR): 9.0
